# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [29]:
import re
import pandas as pd
import numpy as np

#### Open the ` Employee.csv` file and name your dataset `employee`.

In [2]:
employee = pd.read_csv('Employee.csv')
employee

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [3]:
display(employee.groupby(['Department'])['Salary'].mean())

Department
HR       45.00
IT       48.75
Sales    55.00
Name: Salary, dtype: float64

In [4]:
display(employee.pivot_table(index=['Department'], values = 'Salary').reset_index())

,Department,Salary
0,HR,45.00
1,IT,48.75
2,Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [5]:
#Used a groupby function instead
employee.pivot_table(index= ['Department','Title'],values = ['Salary'],aggfunc = ['mean'])

mean
                     Salary
Department Title           
HR         VP          70.0
           analyst     32.5
IT         VP          70.0
           analyst     32.5
           associate   60.0
Sales      associate   55.0

#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [6]:
df1 = employee.pivot_table(index= ['Title'], values = ['Years', 'Salary'], aggfunc = {'Years':'sum','Salary':'sum'}).reset_index()
df1.columns = ['Title','Total Salary','Total Years']
df1.head()

,Title,Total Salary,Total Years
0,VP,140,16
1,analyst,130,9
2,associate,170,12


#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [7]:
df2 = employee.pivot_table(index= ['Department','Title'], values = ['Salary', 'Name'], aggfunc = {'Salary':'mean','Name':pd.Series.nunique}).reset_index()
df2.columns = ['Department','Title','No. Employees','Average Salary',]
df2.head()

,Department,Title,No. Employees,Average Salary
0,HR,VP,1,70.0
1,HR,analyst,2,32.5
2,IT,VP,1,70.0
3,IT,analyst,2,32.5
4,IT,associate,1,60.0


#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [11]:
df3 = employee.pivot_table(index= ['Department','Title'], values = ['Salary'], aggfunc = ['median','min','max','std']).reset_index()
df3.columns = ['Department','Title','Salary Median','Minimum Salary','Maximum Salary','Standard Deviation']
df3.head()

,Department,Title,Salary Median,Minimum Salary,Maximum Salary,Standard Deviation
0,HR,VP,70.0,70,70,NaN
1,HR,analyst,32.5,30,35,3.535534
2,IT,VP,70.0,70,70,NaN
3,IT,analyst,32.5,30,35,3.535534
4,IT,associate,60.0,60,60,NaN


#### Based on your comments, fill the missing values with an appropriate value.

In [19]:
# When calculating the std you calculate the distance from the mean and subtract, and you can't get the square root of 0
# Since there is 1 employee, decided to change the NaN values to 0 instead
# The smaller the std, the more normally things are distributed
#df3 = employee.pivot_table(index= ['Department','Title'], values = ['Salary'], aggfunc = {'median','min','max','std'}).reset_index()
df3 = round(df3.fillna(0),2)
df3.head(10)

,Department,Title,Salary Median,Minimum Salary,Maximum Salary,Standard Deviation
0,HR,VP,70.0,70,70,0.00
1,HR,analyst,32.5,30,35,3.54
2,IT,VP,70.0,70,70,0.00
3,IT,analyst,32.5,30,35,3.54
4,IT,associate,60.0,60,60,0.00
5,Sales,associate,55.0,55,55,0.00


#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [15]:
df4 = employee.pivot_table(index= ['Department'], values = ['Salary', 'Name'], aggfunc = {'Salary':'sum','Name':pd.Series.nunique}).reset_index()
df4.columns = ['Department','No of Employees','Salary Spent']
df4.head()

,Department,No of Employees,Salary Spent
0,HR,3,135
1,IT,4,195
2,Sales,2,110


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [17]:
df5 = employee.pivot_table(index= ['Department','Title'], values = ['Years', 'Salary'], aggfunc = {'Years':'sum','Salary':'mean'}).reset_index()
df5.columns = ['Department','Title','Average Salary','Total Years']
df5.head()

,Department,Title,Average Salary,Total Years
0,HR,VP,70.0,8
1,HR,analyst,32.5,6
2,IT,VP,70.0,8
3,IT,analyst,32.5,3
4,IT,associate,60.0,7


### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [20]:
employee.head(15)

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


In [31]:
def mean_no_max(values):
        values = sorted(values)
        values = values[:len(values)-1]
        mean = np.mean(values)
        return mean
df_mean_no_max = employee.pivot_table(index=['Department'],values= ['Salary'], aggfunc = ['mean',mean_no_max])
df_mean_no_max

,mean,mean_no_max
,Salary,Salary
Department,,
HR,45.00,32.500000
IT,48.75,41.666667
Sales,55.00,55.000000
